In [156]:
## reference: https://colab.research.google.com/drive/1QhSnbh-WJVGZjQJF8u974msOL_vAgMeS#scrollTo=PlAGuj5kuZm9
## https://github.com/eisenjulian/nlp_estimator_tutorial/blob/master/nlp_estimators.ipynb

import pandas as pd
import numpy as np

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn import feature_selection
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVR
from sklearn.svm import SVC

from keras.preprocessing.text import Tokenizer

import tensorflow as tf
from tensorboard import summary as summary_lib
from tensorflow.python.keras.preprocessing import sequence

import seaborn as sns
import matplotlib.pyplot as plt

import tempfile
import os
print(tf.__version__)


dir='/Users/xinwang/ai/dataset/kaggle/DonorsChoose/'
train_file='train.csv'
resource_file='resources.csv'
SEED=1000
positive_sample_size=30000

all_train_df = pd.read_csv(dir + train_file)
resource_df = pd.read_csv(dir + resource_file)
label = LabelEncoder()
low_memory=False


def sampleData():
    train_label_1_df = all_train_df.loc[all_train_df['project_is_approved']==1].sample(n=positive_sample_size,
                                                                           random_state=SEED)
    train_label_0_df = all_train_df[all_train_df['project_is_approved']==0]

    train_df = pd.concat([train_label_1_df,train_label_0_df])
    train_df = shuffle(train_df)

    train_df.fillna(value={"teacher_prefix":'Mr.'}, inplace=True)

    return train_df

train_data = sampleData() 

target = 'project_is_approved'
model_dir = tempfile.mkdtemp()


train_data.info()

1.9.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 57734 entries, 50618 to 124032
Data columns (total 16 columns):
id                                              57734 non-null object
teacher_id                                      57734 non-null object
teacher_prefix                                  57734 non-null object
school_state                                    57734 non-null object
project_submitted_datetime                      57734 non-null object
project_grade_category                          57734 non-null object
project_subject_categories                      57734 non-null object
project_subject_subcategories                   57734 non-null object
project_title                                   57734 non-null object
project_essay_1                                 57734 non-null object
project_essay_2                                 57734 non-null object
project_essay_3                                 1922 non-null object
project_essay_4                          

In [157]:
train_data.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
50618,p175427,899658d07db434af27530d8717911408,Mrs.,SC,2017-04-27 08:44:39,Grades 3-5,Music & The Arts,Performing Arts,Mic Check!,"We are a Title I school of approximately 900 students and we are continuing to grow! At our school, we value student diversity and celebrate the leader in each of our students. We were recently awarded Lighthouse Status as a \""Leader In Me\"" School and have received many awards for closing the achievement gap and raising test scores. We are working towards creating a world class Fine Arts program for our students, so they can find their voice and explore their creativity.","An important part of a student's music education experience is performance. At our school, we try to give students the chance to perform as often as possible. Our students are leaders, they love to be in the spotlight! There is nothing we enjoy more than handing our students a microphone and letting them shine! We are limited, however, because we do not have a portable sound system. Without one, our student performers can't be heard and we can't play accompaniment tracks. The Fender Passport Event Portable Sound System is a good quality system that would provide us the opportunity to perform for audiences anywhere!",NaN,NaN,My students need a portable sound system for school and community events.,20,1
23607,p071731,837c49d9f73efe335f712d76cd70f2a7,Mrs.,CA,2017-03-27 12:33:09,Grades 6-8,"Literacy & Language, Special Needs","Literacy, Special Needs",Headphones for Hearing,"The students in my classroom are students with a wide range of behavioral and learning disabilities. My classes serves students with a range of disabilities, including intellectual disabilities, emotional disturbance, autism spectrum disorder and specific learning disabilities. It supports students who struggle with grasping the academics in a general education setting. Typically this means the world comes to fast at them, or sideways, and they just need my classroom to slow things down to give them time to understand. How can we close the achievement gap when students do not have the necessary materials to succeed?","Headphones are a vital tool in our classroom. In our classroom, we use Chromebooks every day and often have activities that involved listening to stories or websites or videos. This means students need headphones. Without headphones, it becomes a cacophony of sound and students would struggle to listen to their own activity. \r\nWe also use Time for Kids magazine every week. It's a great tool to look at and analyze nonfiction text. It also makes current events interesting and relevant to students. \r\nThe donations for this project will improve my classroom by providing the appropriate tools for student learning. It will allow the instruction to be differentiated and students to be pushed to their furthest academic abilities.",NaN,NaN,My students need headphone to use with their Chromebooks and Time for Kids Magazines.,11,1
24426,p017185,3f2ecadf1bade0ac288c20a9cd0e142d,Mrs.,LA,2016-09-01 10:36:29,Grades 9-12,Music & The Arts,"Performing Arts, Visual Arts",Building the Theater: Lessons on Drafting and Theatrical Design,"In a society where the arts are slowly falling away as an \""important\"" part of curriculum, it can be difficult for students to find means of expressing themselves, and for students who have a passion for the arts, options are very limited.\r\n\r\nTheater is an art form perfectly designed to help our students grow, and it gives them a safe and fun means of expressing themselves. My student are lucky enough that we have a theater program, but they could grow even more with better funding. \r\n\r\nThese students work v

In [158]:
train_data[['teacher_prefix', 'project_is_approved']].groupby('teacher_prefix', as_index=False).mean()

,teacher_prefix,project_is_approved
0,Dr.,0.375000
1,Mr.,0.504878
2,Mrs.,0.534025
3,Ms.,0.509251
4,Teacher,0.427046


In [159]:
train_data[['school_state','project_is_approved']].groupby('school_state', as_index=False).mean().sort_values(
    by='project_is_approved', ascending=False)[0:10]

,school_state,project_is_approved
8,DE,0.597484
6,CT,0.582649
41,SD,0.572327
28,ND,0.565789
40,SC,0.564371
35,OH,0.560694
21,ME,0.555133
49,WV,0.554217
47,WA,0.553339
19,MA,0.549643


In [160]:
train_data[['project_grade_category','project_is_approved']].groupby(
    'project_grade_category', as_index=False).mean().sort_values(by='project_is_approved', ascending=False)


,project_grade_category,project_is_approved
0,Grades 3-5,0.533968
3,Grades PreK-2,0.517677
1,Grades 6-8,0.511401
2,Grades 9-12,0.493197


In [161]:
train_data[['project_subject_categories','project_is_approved']].groupby(
    'project_subject_categories', as_index=False).mean().sort_values(
    by='project_is_approved', ascending=False)[0:10]

,project_subject_categories,project_is_approved
50,"Warmth, Care & Hunger",0.705680
15,"Health & Sports, Warmth, Care & Hunger",0.666667
31,"Literacy & Language, Warmth, Care & Hunger",0.636364
27,"Literacy & Language, History & Civics",0.610811
18,"History & Civics, Health & Sports",0.600000
19,"History & Civics, Literacy & Language",0.591264
35,"Math & Science, History & Civics",0.574586
25,"Literacy & Language, Applied Learning",0.568047
24,Literacy & Language,0.562136
28,"Literacy & Language, Math & Science",0.560049


In [162]:
train_data[['project_subject_subcategories','project_is_approved']].groupby('project_subject_subcategories', as_index=False).mean().sort_values(
    'project_is_approved', ascending=False)[0:50]


,project_subject_subcategories,project_is_approved
222,"Financial Literacy, Health & Wellness",1.000000
151,"Early Development, Foreign Languages",1.000000
206,"Extracurricular, History & Geography",1.000000
260,"Gym & Fitness, Warmth, Care & Hunger",1.000000
198,"Environmental Science, Team Sports",1.000000
267,"Health & Life Science, Music",1.000000
182,"Environmental Science, Financial Literacy",1.000000
174,"Economics, Literature & Writing",1.000000
171,"Economics, Health & Life Science",1.000000
48,"Character Education, Nutrition Education",1.000000


In [163]:
## much more effective that individual 
cross_teacher_state = train_data[['teacher_prefix','school_state','project_is_approved']].groupby(
    by=['teacher_prefix','school_state'], as_index=False).mean().sort_values(
    'project_is_approved', ascending=False)

# print(cross_teacher_state)
cross_teacher_state[0:10]

,teacher_prefix,school_state,project_is_approved
0,Dr.,CA,1.000000
3,Dr.,NJ,1.000000
1,Dr.,IA,1.000000
159,Ms.,WY,0.777778
35,Mr.,ND,0.750000
208,Teacher,WV,0.714286
23,Mr.,KS,0.714286
19,Mr.,IA,0.695652
181,Teacher,ME,0.666667
28,Mr.,ME,0.652174


In [164]:
## much more effective that individual 
train_data[['school_state','project_grade_category','project_is_approved']].groupby(
    by=['school_state','project_grade_category'], as_index=False).mean().sort_values(
    'project_is_approved', ascending=False)[0:10]

,school_state,project_grade_category,project_is_approved
186,VT,Grades 9-12,0.750000
33,DE,Grades 6-8,0.750000
85,ME,Grades 6-8,0.722222
164,SD,Grades 3-5,0.672414
118,NE,Grades 9-12,0.666667
122,NH,Grades 9-12,0.652174
201,WY,Grades 6-8,0.642857
35,DE,Grades PreK-2,0.625000
27,CT,Grades PreK-2,0.620482
142,OH,Grades 9-12,0.616822


In [165]:
train_data.head()

train_data[['project_subject_categories','project_subject_subcategories','project_is_approved']].groupby(
    by=['project_subject_categories','project_subject_subcategories'], as_index=False).mean().sort_values(
    'project_is_approved', ascending=False)[0:30]

,project_subject_categories,project_subject_subcategories,project_is_approved
115,"Applied Learning, Music & The Arts","Parent Involvement, Performing Arts",1.000000
179,History & Civics,"Financial Literacy, Social Sciences",1.000000
190,"History & Civics, Health & Sports","Civics & Government, Health & Wellness",1.000000
194,"History & Civics, Literacy & Language","Civics & Government, Foreign Languages",1.000000
189,"History & Civics, Applied Learning","History & Geography, Parent Involvement",1.000000
331,"Math & Science, History & Civics","Environmental Science, Financial Literacy",1.000000
198,"History & Civics, Literacy & Language","Economics, Literature & Writing",1.000000
47,"Applied Learning, Health & Sports","Other, Team Sports",1.000000
352,"Math & Science, Music & The Arts","Health & Life Science, Music",1.000000
320,"Math & Science, Health & Sports","Environmental Science, Team Sports",1.000000


In [166]:
train_data['num_posted_projects_bins'] = pd.cut(
    train_data['teacher_number_of_previously_posted_projects'],
    [-1,7,50,1000])


train_data[['num_posted_projects_bins','project_is_approved']].groupby(
    'num_posted_projects_bins', as_index=False).mean().sort_values(
    'project_is_approved', ascending=False)

,num_posted_projects_bins,project_is_approved
2,"(50, 1000]",0.722997
1,"(7, 50]",0.583002
0,"(-1, 7]",0.490573


In [167]:
train_data[['project_subject_categories','num_posted_projects_bins','project_is_approved']].groupby(
    by=['project_subject_categories','num_posted_projects_bins'], as_index=False).mean().sort_values(
    'project_is_approved', ascending=False)[0:20]

,project_subject_categories,num_posted_projects_bins,project_is_approved
29,"Health & Sports, Applied Learning","(50, 1000]",1.000000
143,"Special Needs, Health & Sports","(50, 1000]",1.000000
125,"Music & The Arts, Applied Learning","(50, 1000]",1.000000
55,"History & Civics, Health & Sports","(7, 50]",1.000000
131,"Music & The Arts, History & Civics","(50, 1000]",1.000000
47,"Health & Sports, Warmth, Care & Hunger","(50, 1000]",1.000000
46,"Health & Sports, Warmth, Care & Hunger","(7, 50]",1.000000
80,"Literacy & Language, Health & Sports","(50, 1000]",1.000000
94,"Literacy & Language, Warmth, Care & Hunger","(7, 50]",1.000000
95,"Literacy & Language, Warmth, Care & Hunger","(50, 1000]",1.000000


In [168]:
train_data['num_posted_projects_bins'] = pd.cut(
    train_data['teacher_number_of_previously_posted_projects'],
    [-1,7,50,1000])

label = LabelEncoder()

train_data['num_posted_projects_buckets'] = label.fit_transform(train_data['num_posted_projects_bins'])

train_data['num_posted_projects_buckets'].head(10)

50618     1
23607     1
24426     0
67176     0
162414    1
146352    2
19977     0
16996     0
84986     0
113157    0
Name: num_posted_projects_buckets, dtype: int64

In [169]:
resource_file = dir + 'resources.csv'
r_df = pd.read_csv(resource_file)
print(r_df.columns)

total_quantity_df = r_df[['id','quantity']].groupby('id', as_index=False).sum()
total_price_df = r_df[['id','price']].groupby('id', as_index=False).sum()

train_data = pd.merge(train_data, total_quantity_df, how='inner', on='id')
train_data = pd.merge(train_data, total_price_df, how='inner', on='id')

train_data['quantity'].describe()

Index(['id', 'description', 'quantity', 'price'], dtype='object')


count    57734.000000
mean        18.362577
std         27.720532
min          1.000000
25%          4.000000
50%         10.000000
75%         22.000000
max        808.000000
Name: quantity, dtype: float64

In [170]:
train_data['quantity_bins'] = pd.cut(train_data['quantity'], [0,4,10,22,1000])

train_data[['quantity_bins','project_is_approved']].groupby('quantity_bins').mean().sort_values(
    'project_is_approved', ascending=False)

,project_is_approved
quantity_bins,
"(0, 4]",0.646926
"(4, 10]",0.512734
"(22, 1000]",0.475307
"(10, 22]",0.431606


In [171]:
r_df = r_df[0:100000]

r_df.head()

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Box of 96, Yellow (13872)",2,13.59
4,p069063,"EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS (TRANQUIL BLUE), SET OF 4",3,24.95


In [230]:
groups = r_df[['id','description']].groupby('id', as_index=False)


id_column = []
desc_column = []
k=0
for name, group in groups:
    id_column.append(name)

    desc = ' '.join(str(k) if type(k)==int or type(k)==float else k for k in group['description'].values)
        
    desc_column.append(desc)
    k += 1
    
    if k % 500 == 0:
        print(k)
        print(name)
        print(desc)
       
    
desc_df = pd.DataFrame({
    "id":id_column,
    "description": desc_column
})

desc_df.head()

500
p007227
Core Stability Balls

Options: 55 cm - Green
1000
p014806
3D Safari Animal Sand Molds Set - 1 lb Sands Alive, 3 3D Animal Shapes, 2 Rollers And Play Sand Tray 4M Crystal Growing Experiment 4M Kitchen Science Kit Amazing (Mostly) Edible Science: A Family Guide to Fun Experiments in the Kitchen Cranium Family Edition Creativity for Kids Grow 'n Glow Terrarium Curious Chef 17-Piece Measure & Prep Kit Family Pastimes Secret Door - An Award Winning Co-operative Mystery Game Hasbro Pie Face Game HedBanz Game - Edition may vary Modeling Clay Animals: Easy-to-Follow Projects in Simple Steps My First Origami Kit: [Origami Kit with Book, 60 Papers, 150 Stickers, 20 Projects] Peaceable Kingdom Hoot Owl Hoot! Award Winning Cooperative Game for Kids Peaceable Kingdom Stone Soup Award Winning Cooperative Game for Kids Pom-pom Pals: Pets Sago Brothers Ultra Light Modeling Clay Magic Air Dry Clay 24 Colors a Box Scientific Explorer My First Fizzy Foamy Science Kit Scientific Explorer My Fi

14500
p224965
A Big Guy Took My Ball! (An Elephant and Piggie Book) Are You Ready to Play Outside? (An Elephant and Piggie Book) Can I Play Too? (An Elephant and Piggie Book) Elephants Cannot Dance! (An Elephant and Piggie Book) Happy Pig Day! (An Elephant and Piggie Book) I Am Going! (An Elephant and Piggie Book) I Am Invited to a Party! (An Elephant and Piggie Book) I Broke My Trunk! (An Elephant and Piggie Book) I Love My New Toy! (An Elephant and Piggie Book) I Really Like Slop! (An Elephant and Piggie Book) I Will Surprise My Friend! (An Elephant and Piggie Book) I Will Take A Nap! (An Elephant and Piggie Book) I'm a Frog! (An Elephant and Piggie Book) Let's Go for a Drive! (An Elephant and Piggie Book) Listen to My Trumpet! (An Elephant and Piggie Book) My Friend is Sad (An Elephant and Piggie Book) My New Friend Is So Fun! (An Elephant and Piggie Book) Pigs Make Me Sneeze! (An Elephant and Piggie Book) Should I Share My Ice Cream? (An Elephant and Piggie Book) The Thank You Book

,id,description
0,p000027,"BALL INFLATABLE THERAPY AND EXERCISE 65CM BALL INFLATABLE THERAPY AND EXERCISE 75CM BALL INFLATABLE ULTIMAX PUSHBALL 130CM LT.GRN Bouncy Bands for Chair (Blue support pipes) CAPRET 16''X16'' SQUARES JOY IN A BOX SET OF 24 CARPET SQUARES CHAIR PRESCHOOOL HIGH BACK SEATING PLUSH BLUE/RED FLOOR PILLOW 30'' DIA SMALL COLOR WHEEL CUDDLE UPS FLOOR PILLOW 30'' DIA SMALL COLOR WHEEL CUDDLE UPS Kore Patented WOBBLE Chair, Made in the USA, Active Sitting for Toddler, Pre-School, Kids, and Teens; Kids don't have to sit still anymore - ""The BEST seat in any Classroom""! - Blue - Kids (14in) Kore Patented WOBBLE Chair, Made in the USA, Active Sitting for Toddler, Pre-School, Kids, and Teens; Kids don't have to sit still anymore - ""The BEST seat in any Classroom""! - Red - Kids (14in) STOOL STABILI-T T STOOL ROUND 1 LEG"
1,p000031,"4 Year Desktop Protection Plan ($300-350) Lenovo C20 19.5"" AIO Desktop (Celeron, 4GB, 500 GB HDD) F0BB007CUS"
2,p000052,"Homz Plastic Storage Tote Box, With Lid, 10 Gallon, Black and Silver, Stackable, 10-Pack Homz Plastic Storage Tote Box, With Lid, 18 Gallon, Black and Silver, Stackable, 8-Pack"
3,p000069,"Bouncy Bands for Desks Kids Kore Wobble Chair 14"" - Purple Wacces® Fitness Exercise and Stability Ball (Blue, 55 cm) Wacces® Fitness Exercise and Stability Ball (Green, 55 cm)"
4,p000073,Dell Chromebook 11 3120 11 6 Celeron N2840 4 GB RAM 16 GB SSD En Maxell NB 201 headphones


In [232]:
pd.set_option('max_colwidth', 80)
train_data.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,num_posted_projects_bins,num_posted_projects_buckets,quantity,price,quantity_bins
0,p175427,899658d07db434af27530d8717911408,Mrs.,SC,2017-04-27 08:44:39,Grades 3-5,Music & The Arts,Performing Arts,Mic Check!,We are a Title I school of approximately 900 students and we are continuing ...,An important part of a student's music education experience is performance. ...,NaN,NaN,My students need a portable sound system for school and community events.,20,1,"(7, 50]",1,1,699.99,"(0, 4]"
1,p071731,837c49d9f73efe335f712d76cd70f2a7,Mrs.,CA,2017-03-27 12:33:09,Grades 6-8,"Literacy & Language, Special Needs","Literacy, Special Needs",Headphones for Hearing,The students in my classroom are students with a wide range of behavioral an...,"Headphones are a vital tool in our classroom. In our classroom, we use Chrom...",NaN,NaN,My students need headphone to use with their Chromebooks and Time for Kids M...,11,1,"(7, 50]",1,21,24.44,"(10, 22]"
2,p017185,3f2ecadf1bade0ac288c20a9cd0e142d,Mrs.,LA,2016-09-01 10:36:29,Grades 9-12,Music & The Arts,"Performing Arts, Visual Arts",Building the Theater: Lessons on Drafting and Theatrical Design,"In a society where the arts are slowly falling away as an \""important\"" part...",Theater classes are a great means of expression and creation for many studen...,NaN,NaN,"My students need drafting materials such as drafting boards, cutting mats an...",0,1,"(-1, 7]",0,35,268.91,"(22, 1000]"
3,p218324,106c8ca656694a0c0ea47a86d9e6f99b,Teacher,CA,2016-07-15 14:09:33,Grades PreK-2,Literacy & Language,Literacy,iPads Needed for My 21st Century First Graders,My first grade classroom is filled with excitement and a love of learning as...,"Some of the Apps that we will use on our iPads include Raz-Kids, aka Reading...",NaN,NaN,My students need 3 Mini Apple iPads to enhance and support their learning in...,0,1,"(-1, 7]",0,6,328.16,"(4, 10]"
4,p123968,c6f2983c0c2eff430b8a6473a13d2386,Ms.,FL,2017-04-07 11:14:55,Grades 9-12,Literacy & Language,"Literacy, Literature & Writing",Tools of the Trade!,I teach at a second chance school for students who are trying to keep up wit...,My students often come to school without even a pencil. Making sure they hav...,NaN,NaN,"My students need basic supplies such as notebooks, pencils, folders, and hig...",25,0,"(7, 50]",1,224,59.22,"(22, 1000]"


In [234]:
train_data['posted_projects_bins'] = label.fit_transform(train_data['num_posted_projects_bins'])
train_data['quantity_bins'] = label.fit_transform(train_data['quantity_bins'])


In [241]:
print(train_data['quantity_bins'].max())

print(train_data['posted_projects_bins'].max())


3
2


In [245]:
def mix_operation(a, b):
    return str(a) + '_' + str(b)

train_data['posted_mix_quantity'] = train_data[['quantity_bins','posted_projects_bins']].apply(
    lambda row: mix_operation(row['quantity_bins'], row['posted_projects_bins']), 
    axis=1)

train_data['posted_mix_quantity_bins'] = label.fit_transform(train_data['posted_mix_quantity'])

train_data.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,num_posted_projects_bins,num_posted_projects_buckets,quantity,price,quantity_bins,posted_projects_bins,posted_mix_quantity,posted_mix_quantity_bins
0,p175427,899658d07db434af27530d8717911408,Mrs.,SC,2017-04-27 08:44:39,Grades 3-5,Music & The Arts,Performing Arts,Mic Check!,We are a Title I school of approximately 900 students and we are continuing ...,An important part of a student's music education experience is performance. ...,NaN,NaN,My students need a portable sound system for school and community events.,20,1,"(7, 50]",1,1,699.99,0,1,0_1,1
1,p071731,837c49d9f73efe335f712d76cd70f2a7,Mrs.,CA,2017-03-27 12:33:09,Grades 6-8,"Literacy & Language, Special Needs","Literacy, Special Needs",Headphones for Hearing,The students in my classroom are students with a wide range of behavioral an...,"Headphones are a vital tool in our classroom. In our classroom, we use Chrom...",NaN,NaN,My students need headphone to use with their Chromebooks and Time for Kids M...,11,1,"(7, 50]",1,21,24.44,2,1,2_1,7
2,p017185,3f2ecadf1bade0ac288c20a9cd0e142d,Mrs.,LA,2016-09-01 10:36:29,Grades 9-12,Music & The Arts,"Performing Arts, Visual Arts",Building the Theater: Lessons on Drafting and Theatrical Design,"In a society where the arts are slowly falling away as an \""important\"" part...",Theater classes are a great means of expression and creation for many studen...,NaN,NaN,"My students need drafting materials such as drafting boards, cutting mats an...",0,1,"(-1, 7]",0,35,268.91,3,0,3_0,9
3,p218324,106c8ca656694a0c0ea47a86d9e6f99b,Teacher,CA,2016-07-15 14:09:33,Grades PreK-2,Literacy & Language,Literacy,iPads Needed for My 21st Century First Graders,My first grade classroom is filled with excitement and a love of learning as...,"Some of the Apps that we will use on our iPads include Raz-Kids, aka Reading...",NaN,NaN,My students need 3 Mini Apple iPads to enhance and support their learning in...,0,1,"(-1, 7]",0,6,328.16,1,0,1_0,3
4,p123968,c6f2983c0c2eff430b8a6473a13d2386,Ms.,FL,2017-04-07 11:14:55,Grades 9-12,Literacy & Language,"Literacy, Literature & Writing",Tools of the Trade!,I teach at a second chance school for students who are trying to keep up wit...,My students often come to school without even a pencil. Making sure they hav...,NaN,NaN,"My students need basic supplies such as notebooks, pencils, folders, and hig...",25,0,"(7, 50]",1,224,59.22,3,1,3_1,10
